In [13]:
###pip install pandas sqlalchemy pymysql

In [1]:
# HTTP requests to get data from an API
import requests
# Data manipulation and analysis.
import pandas as pd
# maybe
import numpy as np
# Represent dates
import datetime
#json manipulation
import json
#directory management
import os
#security issues
from sqlalchemy import create_engine


# Print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Print all of the data in a feature
pd.set_option('display.max_colwidth', None)

In [2]:
#Leer credenciales
host = os.getenv("MYSQL_host")
port = '3306'
user = os.getenv("MYSQL_user")
password = os.getenv("MYSQL_contra")
database = os.getenv("MYSQL_DB")
inegi=os.getenv("inegi_key")

<h3>Seleccion de municipios a extraer<h3>

Se requiere crear un df con el nombre de los municipios asi como sus claves para usar la API despues y extraer especificamente la localidad que buscamos.

In [3]:
#create dataset with location codes
pre={'municipio': ['Apodaca', 'Cadereyta Jimenez', 'El Carmen', 'Garcia','San Pedro Garza Garcia','General Escobedo',
                   'Guadalupe','Juarez','Monterrey','Salinas Victoria','San Nicolas de los Garza','Santa Catarina','Santiago'],
        'code': ['070000190006', '070000190009', '070000190010', '070000190018','070000190019','070000190021',
                 '070000190026','070000190031','070000190039','070000190045','070000190046','070000190048','070000190049']}

claves = pd.DataFrame(pre)
print(claves)

                   municipio          code
0                    Apodaca  070000190006
1          Cadereyta Jimenez  070000190009
2                  El Carmen  070000190010
3                     Garcia  070000190018
4     San Pedro Garza Garcia  070000190019
5           General Escobedo  070000190021
6                  Guadalupe  070000190026
7                     Juarez  070000190031
8                  Monterrey  070000190039
9           Salinas Victoria  070000190045
10  San Nicolas de los Garza  070000190046
11            Santa Catarina  070000190048
12                  Santiago  070000190049


URL base que usamos para posteriormente modificar el segmento correspondiente a la entidad geografica a analizar "700".

In [42]:
url_base= (f'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/6200240329/es/07000019/false/BISE/2.0/{inegi}?type=json')

Reemplazamos el valor de la URL base con el metodo replace y un f-string para hacer una lista con las URLs de cada municipio del area metropolitana de N.L. 

In [37]:
urls_actualizadas = [
    url_base.replace("0700", code) for code in claves['code']
]

In [43]:
##Practica para evaluar que era una lista y obtener una idea del esqueleto del JSON
response = requests.get(url_base)
response.content

b'["ErrorInfo:No se encontraron resultados","ErrorDetails:No se encontraron resultados","ErrorCode:100"]'

Realizar una funcion para acortar pasos. En donde se convierte el json en un diccionario del cual nos adentramos a 'series' y luego al apartado 'OBSERVATIONS' del cual se genera una variable. De la anterior variable se identifican las etiquetas de nuestro interes para obtener los valores que buscamos y se mandan a una lista llamada 'extracted_data'.

In [6]:
def procesar_json(json_data):
    try:
        observations = json_data.get("Series", [])[0].get("OBSERVATIONS", [])
        for obs in observations:
            extracted_data.append({
                "TIME_PERIOD": obs["TIME_PERIOD"],
                "OBS_VALUE": obs["OBS_VALUE"],
                "COBER_GEO": obs["COBER_GEO"]
            })
    except Exception as e:
        print(f"Error procesando JSON: {e}")

Creación de extracted_data global: Se almacena fuera de la funcion para permitir que los datos procesados se acumulen y se combinen en un solo DataFrame. Se añade un for loop para hacer cada request de la lista de los URLs. Y se convierte todo lo obtenido al final de la funcion en un dataframe.

In [7]:
extracted_data = []
for i in range(len(urls_actualizadas)):
    try:
        # Realizar la solicitud GET
        response = requests.get(urls_actualizadas[i])
        print(f"Solicitud exitosa para: {response.status_code}")
        procesar_json(response.json())
    except requests.exceptions.RequestException as e:
        # Posibles errores de conexión Checar luego como hacerlo mas fino
        print(f"Error en la solicitud para {i}: {e}")
poblacion_total= pd.DataFrame(extracted_data)

Solicitud exitosa para: 200
Solicitud exitosa para: 200
Solicitud exitosa para: 200
Solicitud exitosa para: 200
Solicitud exitosa para: 200
Solicitud exitosa para: 200
Solicitud exitosa para: 200
Solicitud exitosa para: 200
Solicitud exitosa para: 200
Solicitud exitosa para: 200
Solicitud exitosa para: 200
Solicitud exitosa para: 200
Solicitud exitosa para: 200


Cambio de nombre de headers del dataframe 

In [8]:
poblacion_total.rename(columns={'TIME_PERIOD': 'Year', 'OBS_VALUE': 'Poblacion_total','COBER_GEO': 'code'}, inplace=True)

Checar el tipo de datos del dataframe para posteriormente cambiarlos a uno adecuado para el analisis y posterior carga a la base de datos

In [9]:
poblacion_total.dtypes

Year               object
Poblacion_total    object
code               object
dtype: object

In [10]:
# Manejar valores nulos o incorrectos antes de la conversión
poblacion_total['Poblacion_total'] = pd.to_numeric(poblacion_total['Poblacion_total'], errors='coerce')

# Llenar valores nulos con 0 o eliminarlos según sea necesario
poblacion_total['Poblacion_total'].fillna(0, inplace=True)

# Convertir a int
poblacion_total = poblacion_total.astype({
    'Poblacion_total': 'float',
    'code': 'string',
    'Year': 'string'
})

In [22]:
#Ver los valores del df
poblacion_total

,Year,Poblacion_total,code
0,1995,219153.0,070000190006
1,2000,283497.0,070000190006
2,2005,418784.0,070000190006
3,2010,523370.0,070000190006
4,2020,656464.0,070000190006
...,...,...,...
60,1995,34187.0,070000190049
61,2000,36812.0,070000190049
62,2005,37886.0,070000190049
63,2010,40469.0,070000190049


Conexion a MySQL usando credenciales previas y seleccion de la tabla a donde cargar el archivo. En este caso la tabla ya estaba creada y definida.

In [2]:
# Crear conexión a MySQL
try:
    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')
    print("Conexión a MySQL exitosa.")
except Exception as e:
    print(f"Error al conectar a MySQL: {e}")

# Subir el DataFrame al servidor MySQL
try:
    table_name = 'Ingreso_trimestral_hogar_NL'
    Ingreso_trim_hogar_NL.to_sql(table_name, con=engine, if_exists='replace', index=False)
    print(f"Datos subidos exitosamente a la tabla '{table_name}'.")
except Exception as e:
    print(f"Error al subir datos: {e}")

'\n# Crear conexión a MySQL\ntry:\n    engine = create_engine(f\'mysql+pymysql://{user}:{password}@{host}:{port}/{database}\')\n    print("Conexión a MySQL exitosa.")\nexcept Exception as e:\n    print(f"Error al conectar a MySQL: {e}")\n\n# Subir el DataFrame al servidor MySQL\ntry:\n    table_name = \'Ingreso_trimestral_hogar_NL\'\n    Ingreso_trim_hogar_NL.to_sql(table_name, con=engine, if_exists=\'replace\', index=False)\n    print(f"Datos subidos exitosamente a la tabla \'{table_name}\'.")\nexcept Exception as e:\n    print(f"Error al subir datos: {e}")\n    '

Guardado del dataframe en formato csv en caso de usar IA generativa.

In [6]:
Gasto_trim_transp_NL.to_csv("Gasto_trim_transp_NL.csv", index=False)

<h3>Recopilacion de otros datos relevantes para analisis<h3>
Debido a limitantes en accesibilidad usando la API del INEGI, los siguientes datos fueron recopilados de manera manual

**Ingreso corriente total promedio trimestral por hogar en NL (ingreso por trabajo):**
Se realizo una recopilacion de este indicador en el buscador utilizando el siguiente termino: "Ingreso corriente total promedio trimestral por hogar, nuevo leon [año a consultar]". Los datos obtenidos proceden de reportes del INEGI que utilizan la Encuesta Nacional de Ingresos y Gastos de los Hogares(ENIGH).

*NOTA1:* Se considera el ingreso por trabajo para este analisis puesto que una buena parte de la gente que se transporta en el sistema publico corresponde a la clase trabajadora

*NOTA2:* Para el año 2020 se tomo un promedio de los dos valores disponibles en los reportes, de igual manera se determino el valor del año 2016 en base al ingreso total promedio trimestral y al valor promedio de las tasas de variacion entre el ingreso total promedio trimestral de cada año y su respectivo ingreso por trabajo.

Las URLS de los reportes que se utilizaron son:
- [ENIGH2016](https://www.inegi.org.mx/contenidos/programas/enigh/nc/2016/doc/presentacion_resultados_enigh2016.pdf)
- [ENIGH2020](https://www.inegi.org.mx/contenidos/programas/enigh/nc/2020/doc/enigh2020_ns_presentacion_resultados_nl.pdf)
- [ENIGH2022-interact](https://www.inegi.org.mx/app/tabulados/interactivos/?pxq=Hogares_Hogares_10_8a5eaa85-f8a0-41f2-ae0a-cdc75a171b32&idrt=54&opc=t)

In [3]:
#Creacion del df
recop={'year': ['2022', '2020', '2018', '2016'],
        'ingreso': ['58798', '47855', '49484', '57324.90054']}

Ingreso_trim_hogar_NL = pd.DataFrame(recop)

**Gasto corriente monetario promedio trimestral por hogar en transporte publico, NL:** Se realizo una recopilacion de este indicador usando los reportes de inegi anteriores y algunos incluidos a continuacion.
- [ENIGH2022-reporte](https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/doc/enigh2022_ns_presentacion_resultados_nl.pdf)

In [4]:
#Creacion del df
recop2={'year': ['2022', '2020', '2018', '2016'],
        'gasto': ['1642', '1431', '2320', '1469']}

Gasto_trim_transp_NL = pd.DataFrame(recop2)

**TranporteNL_inegi:**
Existen datos respecto al transporte urbano de monterrey en el sitio del INEGI. Se recopilaron los datos de transporte urbano de pasajeros de la ciudad de Monterrey, abarcando los sistemas de metro, ecovia, transmetro y metrobus. Los filtros utilizados para la consulta involucran todas las variables con excepcion de numero de rutas para el caso de ecovia y metro, todos los meses del año y el periodo que abarca de 2020-2024. *Sitio donde fueron recopilados:*[Transporte Urbano de Pasajeros INEGI](https://www.inegi.org.mx/programas/transporteurbano/#tabulados)

Se descargaron archivos .xlxs de cada una de las consultas hechas. Posteriormente, todos los datos fueron compilados en un solo archivo .xlxs para su posterior limpieza.

**NOTA:** Las lineas de codigo a continuacion consideran que ya se realizaron las predicciones de los valores faltantes, asi como, la limpieza del conjunto de datos de TransporteNL_inegi. Los pasos a continuacion solo funcionaron para actualizar las tablas con valores predichos asi como las faltantes al servidor de MySQL

In [9]:
##subir otros dataset a mysql
pop = pd.read_csv("Gasto_trim_transp_NL_modeled.csv")

In [10]:
# Crear conexión a MySQL
try:
    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')
    print("Conexión a MySQL exitosa.")
except Exception as e:
    print(f"Error al conectar a MySQL: {e}")

# Subir el DataFrame al servidor MySQL
try:
    table_name = 'gasto_trim_transp_nl'
    pop.to_sql(table_name, con=engine, if_exists='replace', index=False)
    print(f"Datos subidos exitosamente a la tabla '{table_name}'.")
except Exception as e:
    print(f"Error al subir datos: {e}")

Conexión a MySQL exitosa.
Datos subidos exitosamente a la tabla 'gasto_trim_transp_nl'.
